# Designing and Creating a Database

We will use data from Retrosheet on Major League Baseball statistics to create several tables in a database using SQL.

Included here are the main file, game_log.csv, and several helper files, park_codes.csv, person_codes.csv, and team_codes.csv.  The file game_log_fields.txt explains each of the fields that are included in the main file.

We will begin with some exploratory data analysis.

In [1]:
# imports
import sqlite3
import pandas as pd
import csv

# set options to avoid truncation, as the main file is big
pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

In [2]:
log = pd.read_csv('game_log.csv',low_memory=False)
print(log.shape)
log.head()

(171907, 161)


,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,NaN,1,FW1,NaN,1,0,2,54.0,D,NaN,NaN,NaN,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,6.0,1.0,NaN,-1.0,NaN,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,NaN,-1.0,NaN,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,NaN,NaN,NaN,NaN,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,NaN,1,WS3,NaN,1,20,18,54.0,D,NaN,NaN,NaN,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,NaN,5.0,3.0,NaN,-1.0,NaN,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0,49.0,14.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,10.0,NaN,2.0,1.0,NaN,-1.0,NaN,14.0,1.0,7.0,7.0,0.0,0.0,27.0,20.0,10.0,2.0,3.0,0.0,dobsh901,Henry Dobson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,NaN,NaN,NaN,NaN,spala101,Al Spalding,braia102,Asa Brainard,wrigg101,George Wright,6.0,barnr102,Ross

In [3]:
log.tail()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
171902,20161002,0,Sun,MIL,NL,162,COL,NL,162,6,4,60.0,D,NaN,NaN,NaN,DEN02,27762.0,203.0,0200000202,1100100010,39.0,10.0,4.0,1.0,2.0,6.0,0.0,0.0,1.0,4.0,0.0,12.0,2.0,1.0,0.0,0.0,8.0,7.0,4.0,4.0,1.0,0.0,30.0,12.0,1.0,0.0,0.0,0.0,41.0,13.0,4.0,0.0,1.0,4.0,1.0,0.0,1.0,3.0,0.0,11.0,0.0,1.0,0.0,0.0,12.0,5.0,6.0,6.0,0.0,0.0,30.0,13.0,0.0,0.0,0.0,0.0,barrs901,Scott Barry,woodt901,Tom Woodring,randt901,Tony Randazzo,ortir901,Roberto Ortiz,NaN,NaN,NaN,NaN,counc001,Craig Counsell,weisw001,Walt Weiss,thort001,Tyler Thornburg,rusic001,Chris Rusin,knebc001,Corey Knebel,susaa001,Andrew Susac,cravt001,Tyler Cravy,marqg001,German Marquez,villj001,Jonathan Villar,5.0,genns001,Scooter Gennett,4.0,cartc002,Chris Carter,3.0,santd002,Domingo Santana,9.0,pereh001,Hernan Perez,8.0,arcio002,Orlando Arcia,6.0,susaa001,Andrew Susac,2.0,elmoj001,Jake Elmore,7.0,cravt001,Tyler Cravy,1.0,blacc001,Charlie Blackmon,8.0,dahld001,David Dahl,7.0,arenn001,Nolan Arenado,5.0,gonzc001,Carlos Gonzalez,9.0,murpt002,Tom Murphy,2.0,pattj005,Jordan Patterson,3.0,valap001,Pat Valaika,4.0,adamc001,Cristhian Adames,6.0,marqg001,German Marquez,1.0,NaN,Y
171903,20161002,0,Sun,NYN,NL,162,PHI,NL,162,2,5,51.0,D,NaN,NaN,NaN,PHI13,36935.0,159.0,000001100,00100031x,33.0,8.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,9.0,1.0,1.0,1.0,0.0,6.0,6.0,3.0,3.0,0.0,0.0,24.0,12.0,3.0,1.0,2.0,0.0,33.0,10.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,2.0,0.0,3.0,0.0,0.0,2.0,0.0,7.0,5.0,2.0,2.0,0.0,0.0,27.0,7.0,0.0,0.0,1.0,0.0,barkl901,Lance Barksdale,herna901,Angel Hernandez,barrt901,Ted Barrett,littw901,Will Little,NaN,NaN,NaN,NaN,collt801,Terr

There are over 170,000 rows of data.  The data appears to be organized chronologically, starting in 1871 and continuing through 2016.  There are many columns of data, such as:

- general information on the game
- statistics on the home team
- statistics on the visiting team
- information on each player in the game
- information on the umpires for each game
- information on awards for each game

In [4]:
park = pd.read_csv('park_codes.csv')
print(park.shape)
park.head()

(252, 9)


,park_id,name,aka,city,state,start,end,league,notes
0,ALB01,Riverside Park,NaN,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,NaN,Altoona,PA,04/30/1884,05/31/1884,UA,NaN
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,04/19/1966,NaN,AL,NaN
3,ARL01,Arlington Stadium,NaN,Arlington,TX,04/21/1972,10/03/1993,AL,NaN
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,04/11/1994,NaN,AL,NaN


This is a list of all of the ballparks.  It contains information on the name, location, dates used, and the league for the park.

In [5]:
person = pd.read_csv('person_codes.csv')
print(person.shape)
person.head()

(20494, 7)


,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
0,aardd001,Aardsma,David,04/06/2004,NaN,NaN,NaN
1,aaroh101,Aaron,Hank,04/13/1954,NaN,NaN,NaN
2,aarot101,Aaron,Tommie,04/10/1962,NaN,04/06/1979,NaN
3,aased001,Aase,Don,07/26/1977,NaN,NaN,NaN
4,abada001,Abad,Andy,09/10/2001,NaN,NaN,NaN


This is a list of all of the people in the data.  This contains their names, when they first played, as well as when they first managed, coached (such as first base coach), or umpired.

In [6]:
team = pd.read_csv('team_codes.csv')
print(team.shape)
team.head()

(150, 8)


,team_id,league,start,end,city,nickname,franch_id,seq
0,ALT,UA,1884,1884,Altoona,Mountain Cities,ALT,1
1,ARI,NL,1998,0,Arizona,Diamondbacks,ARI,1
2,BFN,NL,1879,1885,Buffalo,Bisons,BFN,1
3,BFP,PL,1890,1890,Buffalo,Bisons,BFP,1
4,BL1,NaN,1872,1874,Baltimore,Canaries,BL1,1


This is a list of all of the teams in the data.  It contains information on the league, the dates of the team, the location, and nicknames.  There is also a franchise id and a sequence variable.

In [8]:
team['franch_id'].value_counts().head(10)

BS1    4
LAA    3
TRN    3
PHA    3
SE1    3
MLA    3
BR3    3
WS2    2
FLO    2
SL2    2
Name: franch_id, dtype: int64

Let's look more closely at BS1, which has four occurrences in `franch_id`.

In [10]:
team[team['franch_id'] == 'BS1']

,team_id,league,start,end,city,nickname,franch_id,seq
21,BS1,NaN,1871,1875,Boston,Braves,BS1,1
22,BSN,NL,1876,1952,Boston,Braves,BS1,2
23,MLN,NL,1953,1965,Milwaukee,Braves,BS1,3
24,ATL,NL,1966,0,Atlanta,Braves,BS1,4


It appears that as a franchise moves around, the `franch_id` remains constant even though the `team_id` may change.  The `seq` field tracks the number of different team rows added to the data.

## Other Data Elements to Explore

The information on the players in the log file is given by defensive position number.  In baseball, the defensive positions are as follows:

1. Pitcher
2. Catcher
3. First Base
4. Second Base
5. Third Base
6. Shortstop
7. Left Field
8. Center Field
9. Right Field

Let's look a little closer at this.

In [12]:
log['v_player_1_def_pos'].value_counts()

8.0     38715
4.0     28458
6.0     22220
7.0     19595
9.0     14965
5.0     11923
3.0      3383
10.0     1062
2.0       515
1.0         2
Name: v_player_1_def_pos, dtype: int64

While there are only 9 recognized positions, there are a number of 10's in the data.  It is not clear if these are for players such as the designated hitter, a pinch runner, a pinch batter, or just unknown.

In [18]:
log[log['v_player_1_def_pos'] == 10].tail(10)

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
171440,20160830,0,Tue,TOR,AL,132,BAL,AL,132,3,5,51.0,N,NaN,NaN,NaN,BAL12,16083.0,154.0,010000200,00003002x,32.0,7.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,2.0,0.0,5.0,1.0,0.0,2.0,0.0,4.0,3.0,5.0,5.0,0.0,0.0,24.0,6.0,0.0,0.0,0.0,0.0,31.0,7.0,1.0,0.0,3.0,5.0,0.0,0.0,1.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,6.0,3.0,3.0,3.0,0.0,0.0,27.0,12.0,1.0,0.0,2.0,0.0,barrs901,Scott Barry,laynj901,Jerry Layne,wendh902,Hunter Wendelstedt,porta901,Alan Porter,NaN,NaN,NaN,NaN,gibbj001,John Gibbons,showb801,Buck Showalter,bracb001,Brad Brach,grilj001,Jason Grilli,britz001,Zach Britton,wietm001,Matt Wieters,happj001,J.A. Happ,jimeu001,Ubaldo Jimenez,bautj002,Jose Bautista,10.0,donaj001,Josh Donaldson,5.0,encae001,Edwin Encarnacion,3.0,martr004,Russell Martin,2.0,tulot001,Troy Tulowitzki,6.0,saunm001,Michael Saunders,9.0,uptob001,Melvin Upton,7.0,pillk001,Kevin Pillar,8.0,travd001,Devon Travis,4.0,pears001,Steve Pearce,7.0,alvap001,Pedro Alvarez,10.0,machm001,Manny Machado,5.0,trumm001,Mark Trumbo,9.0,davic003,Chris Davis,3.0,schoj001,Jonathan Schoop,4.0,wietm001,Matt Wieters,2.0,hardj003,J.J. Hardy,6.0,reimn001,Nolan Reimold,8.0,NaN,Y
171479,20160902,0,Fri,TOR,AL,134,TBA,AL,133,3,8,51.0,N,NaN,NaN,NaN,STP01,12602.0,172.0,200000010,01100240x,33.0,6.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,12.0,0.0,0.0,0.0,0.0,7.0,4.0,6.0,6.0,0.0,0.0,24.0,11.0,2.0,0.0,0.0,0.0,34.0,8.0,2.0,1.0,3.0,8.0,0.0,0.0,1.0,3.0,0.0,7.0,0.0,0.0,0.0,0.0,6.0,6.0,3.0,3.0,0.0,0.0,27.0,9.0,0.0,0.0,0.0,0.0,tumpj901,John Tumpane,onorb901,Brian O'Nora,hobep901,Pat Hoberg,kellj901,Jeff Kellogg,NaN,NaN,NaN,NaN,gibbj001,John Gibbons,cashk00

Reviewing the above and performing some Google searches on the players in defensive position 10, it appears that all of these players played as designated hitters.  In addition, all of the games are American League games, which have the designated hitter (National League games do not).  My best guess is that this position number is used to indicated the designated hitter, usually abbreviated DH.

Let's also look at the different leagues that are represented in the data.  Currently, there are only two leagues - American League (AL) and National League (NL).  However, from our initial data review, it appears that there are also other leagues in our data.

In [19]:
log['h_league'].value_counts()

NL    88867
AL    74712
AA     5039
FL     1243
PL      532
UA      428
Name: h_league, dtype: int64

With some research on Google, I found the following information about other baseball leagues:

- The National League was formed in 1876. (NL)
- The American League was formed in 1901. (AL)
- The American Association existed from 1882 to 1891. (AA)
- The Federal League existed from 1914 to 1915. (FL)
- The Players League existed for 1890 only. (PL)
- The Union Association  existed for 1884 only. (UA)

Let's see if the data in the file matches up with the historical data we researched.

In [20]:
def league_info(league):
    league_games = log[log['h_league'] == league]
    earliest = league_games['date'].min()
    latest = league_games['date'].max()
    print('{} went from {} to {}'.format(league,earliest,latest))

for league in log['h_league'].unique():
    league_info(league)

nan went from nan to nan
NL went from 18760422 to 20161002
AA went from 18820502 to 18911006
UA went from 18840417 to 18841019
PL went from 18900419 to 18901004
AL went from 19010424 to 20161002
FL went from 19140413 to 19151003


## Import Data into SQLite

Next we need to determine a primary key.  From the data dictionary from Retrosheet, they use a 12 character ID, which we can use as our primary key:

- first three characters are the home team code
- the next four characters are the year the game was played
- the next two characters are the month the game was played
- the next two characters are the day the game was played
- the last character indicates a single game (0), first game of a double header (1), or second game of a double header (2)

We will then create tables for each of our dataframes in our new SQLite database, mlb.db.

In [33]:
# create helper functions
DB = 'mlb.db'

def run_query(q):
    with sqlite3.connect(DB) as conn:
        return pd.read_sql(q, conn)

# add enforcement of foreign keys
def run_command(c):
    with sqlite3.connect(DB) as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)

def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)

In [23]:
# create tables for each of our dataframes
tables = {
    'game_log': log,
    'person_codes': person,
    'team_codes': team,
    'park_codes': park
}

with sqlite3.connect(DB) as conn:    
    for name, data in tables.items():
        conn.execute('DROP TABLE IF EXISTS {};'.format(name))
        data.to_sql(name,conn,index=False)

In [24]:
show_tables()

,name,type
0,park_codes,table
1,team_codes,table
2,game_log,table
3,person_codes,table


In [25]:
# create the 12-character game_id column
c1 = '''
ALTER TABLE game_log
ADD COLUMN game_id TEXT;
'''

try:
    run_command(c1)
except:
    pass

In [26]:
# populate game_id as described above
c2 = '''
UPDATE game_log
SET game_id = h_name || date || number_of_game
WHERE game_id IS NULL; 
'''

run_command(c2)

In [27]:
# review results
q1 = '''
SELECT
    game_id,
    h_name,
    date,
    number_of_game
FROM game_log
LIMIT 5;
'''

run_query(q1)

,game_id,h_name,date,number_of_game
0,FW1187105040,FW1,18710504,0
1,WS3187105050,WS3,18710505,0
2,RC1187105060,RC1,18710506,0
3,CH1187105080,CH1,18710508,0
4,TRO187105090,TRO,18710509,0


## Look for Normalization Opportunities

The next step in this process is to look for opportunities to normalize our data.  The below is a list of some of the steps we can perform.

- In `park_codes` the first and last date a game was played can be determined from the `game_log` data and can be removed from here.
- In `person_codes` we can reproduce all of the debut dates from the data in `game_log` so we can remove all of the debut dates from here.
- In `team_codes` the start, end, and sequence columns can all be reproduced from the `game_log` data and can be removed.
- In `game_log` all of the people's names can be removed, as these can be found in the `person_codes` table.
- In `game_log`, all of the offensive and defensive stats have separate columns for the home and visiting team.  We could cut the number of these columns in half by instead have two rows for each game, one for each team.
- In `game_log` we have columns tracking the player for each position.  We can remove these and make a new table to track player appearances.  We can also do something similar for umpires and managers.
- In `game_log` there are several awards columns.  We could move these into their own table or add to the appearance table.

Using https://www.dbdesigner.net/designer I created the schema for our database:

[my_schema](https://github.com/Frizzles7/Dataquest/blob/master/Project_13/my_schema.png)

## Begin Creating Tables

We will begin by first creating the following tables:

- person
- park
- league
- appearance_type

These are the tables without foreign key relations, and our other tables will refer to these tables, which is why we will create them first.

In [29]:
# create person table
c3 = '''
CREATE TABLE IF NOT EXISTS person (
    person_id TEXT PRIMARY KEY,
    first_name TEXT,
    last_name TEXT
);
'''

c4 = '''
INSERT OR IGNORE INTO person
SELECT
    id,
    first,
    last
FROM person_codes;
'''

q2 = '''
SELECT *
FROM person
LIMIT 5;
'''

run_command(c3)
run_command(c4)
run_query(q2)

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad


In [30]:
# create park table
c5 = '''
CREATE TABLE IF NOT EXISTS park (
    park_id TEXT PRIMARY KEY,
    name TEXT,
    nickname TEXT,
    city TEXT,
    state TEXT,
    notes TEXT
);
'''

c6 = '''
INSERT OR IGNORE INTO park
SELECT
    park_id,
    name,
    aka,
    city,
    state,
    notes
FROM park_codes;
'''

q3 = '''
SELECT *
FROM park
LIMIT 5;
'''

run_command(c5)
run_command(c6)
run_query(q3)

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None


In [31]:
# create league table
c7 = '''
CREATE TABLE IF NOT EXISTS league (
    league_id TEXT PRIMARY KEY,
    league_name TEXT
);
'''

c8 = '''
INSERT OR IGNORE INTO league
VALUES
    ('NL', 'National League'),
    ('AL', 'American League'),
    ('AA', 'American Association'),
    ('FL', 'Federal League'),
    ('PL', 'Players League'),
    ('UA', 'Union Association')
;
'''

q4 = '''
SELECT *
FROM league;
'''

run_command(c7)
run_command(c8)
run_query(q4)

,league_id,league_name
0,NL,National League
1,AL,American League
2,AA,American Association
3,FL,Federal League
4,PL,Players League
5,UA,Union Association


In [32]:
# create appearance_type table
c9 = 'DROP TABLE IF EXISTS appearance_type;'

c10 = '''
CREATE TABLE appearance_type (
    appearance_type_id TEXT PRIMARY KEY,
    name TEXT,
    category TEXT
);
'''

run_command(c9)
run_command(c10)

# data for appearance_type is provided in a separate csv file
appearance_type = pd.read_csv('appearance_type.csv')

with sqlite3.connect('mlb.db') as conn:
    appearance_type.to_sql('appearance_type',
                           conn,
                           index=False,
                           if_exists='append') 

q5 = '''
SELECT *
FROM appearance_type;
'''

run_query(q5)

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense
5,O6,Batter 6,offense
6,O7,Batter 7,offense
7,O8,Batter 8,offense
8,O9,Batter 9,offense
9,D1,Pitcher,defense


## Adding Team and Game Tables

The team table is related to the league table, and the game table is related to the park table.  Both of these tables need to be created before the two appearance tables are added, as they refer to these tables.

In [34]:
# create team table
c11 = '''
CREATE TABLE IF NOT EXISTS team (
    team_id TEXT PRIMARY KEY,
    league_id TEXT,
    city TEXT,
    nickname TEXT,
    franch_id TEXT,
    FOREIGN KEY (league_id) REFERENCES league(league_id)
);
'''

c12 = '''
INSERT OR IGNORE INTO team
SELECT
    team_id,
    league,
    city,
    nickname,
    franch_id
FROM team_codes;
'''

q6 = '''
SELECT *
FROM team
LIMIT 5;
'''

run_command(c11)
run_command(c12)
run_query(q6)

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1


In [35]:
# create game table
c13 = '''
CREATE TABLE IF NOT EXISTS game (
    game_id TEXT PRIMARY KEY,
    date TEXT,
    number_of_game INTEGER,
    park_id TEXT,
    length_outs INTEGER,
    day BOOLEAN,
    completion TEXT,
    forefeit TEXT,
    protest TEXT,
    attendance INTEGER,
    legnth_minutes INTEGER,
    additional_info TEXT,
    acquisition_info TEXT,
    FOREIGN KEY (park_id) REFERENCES park(park_id)
);
'''

c14 = '''
INSERT OR IGNORE INTO game
SELECT
    game_id,
    date,
    number_of_game,
    park_id,
    length_outs,
    CASE
        WHEN day_night = "D" THEN 1
        WHEN day_night = "N" THEN 0
        ELSE NULL
        END
        AS day,
    completion,
    forefeit,
    protest,
    attendance,
    length_minutes,
    additional_info,
    acquisition_info
FROM game_log;
'''

q7 = '''
SELECT *
FROM game
LIMIT 5;
'''

run_command(c13)
run_command(c14)
run_query(q7)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forefeit,protest,attendance,legnth_minutes,additional_info,acquisition_info
0,FW1187105040,18710504,0,FOR01,54,1,None,None,None,200,120,None,Y
1,WS3187105050,18710505,0,WAS01,54,1,None,None,None,5000,145,HTBF,Y
2,RC1187105060,18710506,0,RCK01,54,1,None,None,None,1000,140,None,Y
3,CH1187105080,18710508,0,CHI01,54,1,None,None,None,5000,150,None,Y
4,TRO187105090,18710509,0,TRO01,54,1,None,None,None,3250,145,HTBF,Y


## Add team_appearance Table

Next we will add the team_appearance table.  This table has a compound primary key of team_id and game_id, as each game will have two rows, one for the home team and one for the visiting team.

In [36]:
# create team_appearance table
c15 = '''
CREATE TABLE IF NOT EXISTS team_appearance (
    team_id TEXT,
    game_id TEXT,
    home BOOLEAN,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats INTEGER,
    hits INTEGER,
    doubles INTEGER,
    triples INTEGER,
    homeruns INTEGER,
    rbi INTEGER,
    sacrifice_hits INTEGER,
    sacrifice_flies INTEGER,
    hit_by_pitch INTEGER,
    walks INTEGER,
    intentional_walks INTEGER,
    strikeouts INTEGER,
    stolen_bases INTEGER,
    caught_stealing INTEGER,
    grounded_into_double INTEGER,
    first_catcher_interference INTEGER,
    left_on_base INTEGER,
    pitchers_used INTEGER,
    individual_earned_runs INTEGER,
    team_earned_runs INTEGER,
    wild_pitches INTEGER,
    balks INTEGER,
    putouts INTEGER,
    assists INTEGER,
    errors INTEGER,
    passed_balls INTEGER,
    double_plays INTEGER,
    triple_plays INTEGER,
    PRIMARY KEY (team_id, game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id)
);
'''

c16 = '''
INSERT OR IGNORE INTO team_appearance
    SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays
    FROM game_log

UNION

    SELECT
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays
    from game_log;
'''

q8 = '''
SELECT *
FROM team_appearance
LIMIT 5;
'''

run_command(c15)
run_command(c16)
run_query(q8)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,ALT,ALT188404300,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,ALT,ALT188405020,1,UA,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,ALT,ALT188405030,1,UA,5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,ALT,ALT188405050,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,ALT,ALT188405100,1,UA,9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


## Add person_appearance Table

The last table to add is the person_appearance table, which is connected to four of our other tables.

In [37]:
# create person_appearance table
c17 = "DROP TABLE IF EXISTS person_appearance"

c18 = '''
CREATE TABLE person_appearance (
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id,
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
);
'''

c19 = '''
INSERT OR IGNORE INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        NULL,
        hp_umpire_id,
        "UHP"
    FROM game_log
    WHERE hp_umpire_id IS NOT NULL    

UNION

    SELECT
        game_id,
        NULL,
        [1b_umpire_id],
        "U1B"
    FROM game_log
    WHERE "1b_umpire_id" IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [2b_umpire_id],
        "U2B"
    FROM game_log
    WHERE [2b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [3b_umpire_id],
        "U3B"
    FROM game_log
    WHERE [3b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        lf_umpire_id,
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score < v_score THEN h_name
            ELSE v_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE v_starting_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE h_starting_pitcher_id IS NOT NULL;
'''

template = '''
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
'''

run_command(c17)
run_command(c18)
run_command(c19)

for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        run_command(template.format(**query_vars))

In [38]:
q9 = '''
SELECT *
FROM person_appearance
LIMIT 5;
'''

run_query(q9)

,appearance_id,person_id,team_id,game_id,appearance_type_id
0,1,maplb901,None,ALT188404300,UHP
1,2,curte801,ALT,ALT188404300,MM
2,3,murpj104,ALT,ALT188404300,PSP
3,4,hodnc101,SLU,ALT188404300,PSP
4,5,sullt101,SLU,ALT188404300,MM


## Remove Prior Tables

Now that we have our new tables, we can remove the old tables we originally imported.

In [39]:
# view what tables currently exist
show_tables()

,name,type
0,park_codes,table
1,team_codes,table
2,game_log,table
3,person_codes,table
4,person,table
5,park,table
6,league,table
7,appearance_type,table
8,team,table
9,game,table


In [40]:
# drop park_codes, team_codes, game_log, person_codes
tables = ['game_log', 'park_codes', 'team_codes', 'person_codes']

for t in tables:
    c20 = '''
    DROP TABLE {}
    '''.format(t)
    
    run_command(c20)

# check list of tables again
show_tables()

,name,type
0,person,table
1,park,table
2,league,table
3,appearance_type,table
4,team,table
5,game,table
6,team_appearance,table
7,person_appearance,table
